In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [5]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))


In [6]:
df["target_enc_content_id"] = df.groupby("content_id")["answered_correctly"].transform(f)

In [7]:
df["target_enc_user_id"] = df.groupby("user_id")["answered_correctly"].transform(f)

In [9]:
df["user_count"] = df.groupby("user_id").cumcount()

In [10]:
df["user_count_50"] = (df["user_count"] < 50)

In [11]:
df.columns

Index(['timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'bundle_id', 'correct_answer', 'part', 'tags', 'tag', 'type_of',
       'target_enc_content_id', 'target_enc_user_id', 'user_count',
       'user_count_50'],
      dtype='object')

In [13]:
df.groupby(["user_count_50", "prior_question_had_explanation"])["answered_correctly"].describe()

count      mean       std  \
user_count_50 prior_question_had_explanation                                  
False         False                            220782.0  0.648767  0.477357   
              True                            8148070.0  0.675205  0.468298   
True          False                            689690.0  0.450288  0.497523   
              True                             739816.0  0.623959  0.484391   

                                              min  25%  50%  75%  max  
user_count_50 prior_question_had_explanation                           
False         False                           0.0  0.0  1.0  1.0  1.0  
              True                            0.0  0.0  1.0  1.0  1.0  
True          False                           0.0  0.0  0.0  1.0  1.0  
              True                            0.0  0.0  1.0  1.0  1.0